In [1]:
import pandas as pd

In [41]:
# Cấu hình Pandas để hiển thị rộng hơn
pd.set_option('display.width', 1000)  # Tăng chiều rộng hiển thị (thử các giá trị lớn hơn nếu cần)
pd.set_option('display.max_columns', None)  # Hiển thị tất cả các cột
pd.set_option('display.expand_frame_repr', False) # Không xuống dòng DataFrame

def recommend_food(excel_file, calories_prompt_per100=None, ingredient_prompt=None, user_type_prompt=None, taste_prompt=None, negative_prompt=None):
    """
    Đề xuất món ăn từ file Excel dựa trên input của người dùng.

    Args:
        excel_file (str): Đường dẫn đến file Excel chứa dữ liệu món ăn.
        ingredient_prompt (str, optional): Prompt về nguyên liệu mong muốn (ví dụ: "beef, chicken"). Defaults to None.
        user_type_prompt (str, optional): Prompt về loại người dùng (ví dụ: "athlete, normal"). Defaults to None.
        taste_prompt (str, optional): Prompt về hương vị mong muốn (ví dụ: "sweet, smoky"). Defaults to None.
        negative_prompt (dict, optional): Dictionary chứa negative prompts theo loại (ví dụ: {'Ingredient': 'pork', 'Taste': 'sour'}). Defaults to None.

    Returns:
        pandas.DataFrame: DataFrame chứa các món ăn được đề xuất, đã sắp xếp theo điểm ranking.
    """

    # Đọc tất cả các sheet trong file Excel
    excel_data = pd.read_excel(excel_file, sheet_name=None)

    # Kết hợp dữ liệu từ tất cả các sheet thành một DataFrame duy nhất
    df = pd.concat(excel_data.values(), ignore_index=True)

    # Khởi tạo cột 'Ranking Score' với giá trị 0
    df['Ranking Score'] = 0
    
    # Lọc theo Calories/Serving dựa trên chỉ số đầu
    if calories_prompt_per100 is not None:
        calories_first_digit_prompt = str(int(calories_prompt_per100)).split('.')[0][0] # Lấy chỉ số đầu tiên của calories prompt
        df['Calories/Serving_str_first_digit'] = df['Calories/Serving'].astype(str).str[0] # Chuyển cột Calories/Serving sang string và lấy chỉ số đầu
        df = df[df['Calories/Serving_str_first_digit'] == calories_first_digit_prompt] # Lọc các hàng có chỉ số đầu Calories/Serving trùng với prompt
        df = df.drop(columns=['Calories/Serving_str_first_digit']) # Xóa cột phụ trợ

    # Xử lý Positive Prompts
    if ingredient_prompt:
        ingredients = [ing.strip().lower() for ing in ingredient_prompt.split(',')]
        for ingredient in ingredients:
            df.loc[df['Ingredients'].str.lower().str.contains(ingredient, na=False), 'Ranking Score'] += 1
    if user_type_prompt:
        user_types = [ut.strip().lower() for ut in user_type_prompt.split(',')]
        for user_type in user_types:
            df.loc[df['User type'].str.lower().str.contains(user_type, na=False), 'Ranking Score'] += 1
    if taste_prompt:
        tastes = [t.strip().lower() for t in taste_prompt.split(',')]
        for taste in tastes:
            df.loc[df['Taste'].str.lower().str.contains(taste, na=False), 'Ranking Score'] += 1

    # Xử lý Negative Prompts
    if negative_prompt:
        if 'Ingredient' in negative_prompt and negative_prompt['Ingredient']:
            neg_ingredients = [ing.strip().lower() for ing in negative_prompt['Ingredient'].split(',')]
            for neg_ingredient in neg_ingredients:
                df = df[~df['Ingredients'].str.lower().str.contains(neg_ingredient, na=False)] # Loại bỏ hàng chứa negative ingredient
        if 'User Type' in negative_prompt and negative_prompt['User Type']:
            neg_user_types = [ut.strip().lower() for ut in negative_prompt['User Type'].split(',')]
            for neg_user_type in neg_user_types:
                df = df[~df['User type'].str.lower().str.contains(neg_user_type, na=False)] # Loại bỏ hàng chứa negative user type
        if 'Taste' in negative_prompt and negative_prompt['Taste']:
            neg_tastes = [t.strip().lower() for t in negative_prompt['Taste'].split(',')]
            for neg_taste in neg_tastes:
                df = df[~df['Taste'].str.lower().str.contains(neg_taste, na=False)] # Loại bỏ hàng chứa negative taste

    # Sắp xếp theo Ranking Score giảm dần
    ranked_df = df.sort_values(by='Ranking Score', ascending=False).reset_index(drop=True)
    
    columns_to_drop = ['No'] + ['Serving'] + ['Calories']
    ranked_df = ranked_df.drop(columns=columns_to_drop, errors='ignore')

    return ranked_df

In [46]:
excel_file_path = 'food.xlsx'

# Input của người dùng
calories_prompt_per100 = 4.5
user_ingredient_prompt = 'beef, bacon'
user_user_type_prompt = 'athlete'
user_taste_prompt = 'rich, smoky'
user_negative_prompt = {'Taste': 'tender'} 

In [48]:
# Gọi hàm đề xuất
recommended_foods = recommend_food(
    calories_prompt_per100=calories_prompt_per100,
    excel_file=excel_file_path,
    ingredient_prompt=user_ingredient_prompt,
    user_type_prompt=user_user_type_prompt,
    taste_prompt=user_taste_prompt,
    negative_prompt=user_negative_prompt
)

# In ra kết quả
print(recommended_foods)

                     Food             Serving.1 Calories.1  Calories/Serving                 Ingredients              User type                       Taste  Ranking Score
0     Baconator (Wendy's)       1 piece (227 g)   1010 cal              4.45         bacon, beef, cheese          gain, athlete  rich, salty, savory, smoky              5
1       Jerky, Beef Jerky        1 piece (20 g)     91 cal              4.57                        beef          gain, athlete        salty, savory, smoky              3
2         Shredded Cheese       1 portion (5 g)     25 cal              4.96                      cheese        athlete, normal         rich, salty, savory              2
3         Manchego Cheese        1 piece (28 g)    120 cal              4.29  sheep's milk, rennet, salt  gain, athlete, normal         rich, salty, savory              2
4          Gruyere Cheese        1 slice (28 g)    116 cal              4.13              gruyere cheese        athlete, normal  nutty, rich, sav